First, run the command to get the embeddings of the upcoming weeks. We also note this model (to get the embeddings as well as  one first model to learn the embeddings). This model will then be used to genereate embeddings for all upcoming weeks.

In [2]:
day = 20160701
! export CUDA_VISIBLE_DEVICES=3 && python main.py --data real-t --semi_supervised 0 --batch_size 128 --sampling hybrid --subsamplings bATE/DATE --weights 0/1 --mode scratch --train_from 20160101 --test_from $day --test_length 350 --valid_length 30 --initial_inspection_rate 20 --final_inspection_rate 5 --epoch 5 --closs bce --rloss full --save 0 --numweeks 1 --inspection_plan direct_decay

Experiment starts:  1627093784.306
Namespace(act='relu', ada_lr=0.8, agg='sum', alpha=10, batch_size=128, closs='bce', data='real-t', device='0', devices=['0', '1', '2', '3'], dim=16, epoch=5, final_inspection_rate=5.0, fusion='concat', head_num=4, identifier='1627093784.306', initial_inspection_rate=20.0, inspection_plan='direct_decay', l2=0.01, lr=0.005, mode='scratch', numweeks=1, output='result-1627093784.306', rev_func='log', rloss='full', sampling='hybrid', save=0, semi_supervised=0, ssl_strategy='random', subsamplings='bATE/DATE', test_from='20160701', test_length=350, train_from='20160101', uncertainty='naive', use_self=1, valid_length=30, weights='0/1')
Before masking:
 0    373394
1     28939
Name: illicit, dtype: int64
NumExpr defaulting to 6 threads.
After masking:
 0.0    74643
1.0     5824
Name: illicit, dtype: int64
Inspection rate for testing periods: [5.]
Data size:
Train labeled: (80467, 41), Train unlabeled: (321866, 41), Valid labeled: (69932, 41), Valid unlabeled: 

This will store the embeddings in the directory './intermediary/embeddings/embd_0.pickle'
These embeddings will be domain-shift calculated between weeks

Calculate Domain shifts between weeks

In [ ]:
import pandas as pd 
import numpy as np
import torch
import random

import pickle

import ot

In [ ]:
with open("./intermediary/embeddings/embd_0.pickle","rb") as f :
    processed_data = pickle.load(f)

Matching the datapoints together 

In [ ]:
datestart = '16-07-01'

from datetime import datetime, timedelta

date = datetime.strptime(datestart, "%y-%m-%d")

datelist0 = []

for i in range(51):
    
    delta = i * 7
    
    new_date = date + timedelta(days=delta)
    
    datelist0.append(new_date.strftime('%y-%m-%d'))
    
enddate = datelist0[-1]

data = pd.read_csv('tdata.csv')
data = data[(data['sgd.date'] >= datestart) & (data['sgd.date'] < enddate)]

data.reset_index(drop=True, inplace=True)

Concat weekly data together

In [ ]:
full_data = []

for i in range(len(datelist0) - 1):
    
    data0 = data[(data['sgd.date'] >= datelist0[i]) & (data['sgd.date'] < datelist0[i+1])]
    
    start = data0.index[0]
    end = data0.index[-1]
    
    xd = []
    
    for j in range(start, end + 1):
        xd.append(processed_data[datelist[j]].reshape(1, -1))
        
    full_data.append(torch.cat(xd, axis=0))
    

In [ ]:
def domain_shift(xs, xt):
    
    M = torch.cdist(xs, xt)
    
    lol = torch.max(M)

    M = M/lol
    
    M = M.data.cpu().numpy()
    
    a = [1/xs.shape[0]] * xs.shape[0]
    b = [1/xt.shape[0]] * xt.shape[0]
    
    prep = ot.emd2(a, b, M)

    return prep * lol

In [ ]:
arraycom = []
for i in range(81, len(full_data) - 1):
    
    stack = []
    
    for j in range(60):
        
        indices = torch.tensor(random.sample(range(full_data[i].shape[0]), 500))
        indices2 = torch.tensor(random.sample(range(full_data[i+1].shape[0]), 500))
        
        xs = full_data[i][indices]
        xt = full_data[i+1][indices2]
        
        print(domain_shift(xs, xt))
        
        stack.append(domain_shift(xs, xt))
        
    xd = np.mean(stack)
    
    print(xd)
    
    arraycom.append(xd)

arraycom will contain domainshifts information  through 100 weeks

This part is about extracting the best probability of resampling through each week

In [ ]:
def extraction(lol):
    for i in lol:
        if 'Pr@5.0' in i:
            xd = i.split(', ')[0]
            value = float(xd.split(':')[1])
            
    return value

In [ ]:
probs = []
for i in range(0, len(datelist0) - 2):
    
    highest = 0
    day = datelist0[i]
    
    for j in range(1, 10):
        
        ratio = j * 0.1
        bratio =  1 - ratio
        
        lol = ! export CUDA_VISIBLE_DEVICES=3 && python main.py --data real-t --semi_supervised 0 --batch_size 128 --sampling hybrid --subsamplings xgb/random --weights $ratio/$bratio --mode scratch --train_from 20160101 --test_from $day --test_length 7 --valid_length 30 --initial_inspection_rate 20 --final_inspection_rate 5 --epoch 5 --closs bce --rloss full --save 0 --numweeks 2 --inspection_plan direct_decay
        
        value = extract(lol)
        if value > highest:
            smt = bratio
            
    probs.append(smt)

probs should contain the best resampling ratio. Now we come to find the coefficients. First transform the probs result

In [ ]:
news = []
for i in probs:
    news.append(np.log(i/(1-i)))

newcom = np.reshape(arraycom, (-1, 1))


In [ ]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression()

reg.fit(newcom, news)
reg.coef_

In [ ]:
reg.intercept_